# Fase B: Construcción del Dataset a Nivel Producto

**Autor:** Diego Monroy Minero  
**Proyecto:** Segmentación de productos BigMart

## Objetivos de la Notebook
1.  **Limpieza Granular:** Corregir inconsistencias en categorías y tratar valores nulos con lógica de negocio (no imputación ciega).
2.  **Feature Engineering:** Crear categorías amplias y ajustar variables semánticas.
3.  **Agregación (Paso Crítico):** Transformar el dataset de transacciones (Tienda-Producto) a un dataset de entidades (Producto único).
4.  **Preparación para Clustering:** Generar dos versiones de los datos: una legible para humanos (Interpretación) y una numérica escalada para máquinas (Modelado).

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
pd.set_option('display.max_columns', None)

# Carga de datos
# Asumimos que la estructura de carpetas es: Notebooks/fase_b.ipynb y Data/Raw/train.csv
try:
    df = pd.read_csv('../Data/Raw/train.csv')
    print(f"Datos cargados correctamente. Dimensiones iniciales: {df.shape}")
except FileNotFoundError:
    print("Error: No se encontró el archivo. Verifica la ruta relativa.")

Datos cargados correctamente. Dimensiones iniciales: (8523, 12)


--- 
## 1. Limpieza de Datos Granular

Antes de agregar, debemos limpiar a nivel fila para asegurar que los promedios y conteos sean precisos.

### 1.1 Estandarización de Categorías
La columna `Item_Fat_Content` tiene etiquetas inconsistentes ('LF', 'low fat', 'Low Fat').

In [2]:
# Mapeo de corrección
fat_content_map = {
    'low fat': 'Low Fat',
    'LF': 'Low Fat',
    'reg': 'Regular'
}

# Aplicar corrección (respetando los que ya están bien como 'Low Fat' y 'Regular')
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(fat_content_map)

print("Distribución corregida de Fat Content:")
print(df['Item_Fat_Content'].value_counts())

Distribución corregida de Fat Content:
Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64


### 1.2 Tratamiento de Nulos: Lógica de Negocio

**Item_Weight:** Un mismo producto (`Item_Identifier`) debe pesar lo mismo en todas las tiendas. Usaremos esto para imputar.
**Item_Visibility:** Una visibilidad de 0.0 es imposible. La trataremos como nulo y la imputaremos con el promedio de visibilidad de ese producto.

In [3]:
# 1. Tratamiento de Visibilidad
# Reemplazar 0.0 con NaN para que no afecte el cálculo del promedio
df['Item_Visibility'] = df['Item_Visibility'].replace(0, np.nan)

# Imputar la visibilidad con el promedio DE ESE PRODUCTO ESPECÍFICO
df['Item_Visibility'] = df.groupby('Item_Identifier')['Item_Visibility'].transform(lambda x: x.fillna(x.mean()))

# 2. Tratamiento de Peso (Weight)
# Estrategia Primaria: Rellenar con el peso existente del mismo ID
df['Item_Weight'] = df.groupby('Item_Identifier')['Item_Weight'].transform(lambda x: x.fillna(x.mean()))

# Estrategia de Respaldo (Fallback): Si un producto NO tiene peso en ninguna tienda (todos nulos para ese ID),
# imputamos con la media global de su 'Item_Type'
df['Item_Weight'] = df['Item_Weight'].fillna(df.groupby('Item_Type')['Item_Weight'].transform('mean'))

# Verificación final de nulos
print("Nulos restantes tras imputación lógica:")
print(df[['Item_Visibility', 'Item_Weight']].isnull().sum())

Nulos restantes tras imputación lógica:
Item_Visibility    0
Item_Weight        0
dtype: int64


---
## 2. Feature Engineering (Previo a Agregación)

Derivaremos características que son inherentes al producto.

* **Broad_Category:** Las primeras dos letras del ID (FD, DR, NC) nos dicen la categoría macro.
* **Ajuste Semántico:** Si es 'NC' (Non-Consumable), no tiene sentido que tenga 'Low Fat'. Lo cambiaremos a 'Non-Edible'.

In [4]:
# Crear Broad Category
df['Broad_Category'] = df['Item_Identifier'].apply(lambda x: x[:2])
category_map = {'FD': 'Food', 'NC': 'Non-Consumable', 'DR': 'Drinks'}
df['Broad_Category'] = df['Broad_Category'].map(category_map)

# Ajuste lógico: Si es Non-Consumable, Fat_Content = Non-Edible
df.loc[df['Broad_Category'] == 'Non-Consumable', 'Item_Fat_Content'] = 'Non-Edible'

print(pd.crosstab(df['Broad_Category'], df['Item_Fat_Content']))

Item_Fat_Content  Low Fat  Non-Edible  Regular
Broad_Category                                
Drinks                728           0       71
Food                 3190           0     2935
Non-Consumable          0        1599        0


---
## 3. Construcción del Dataset Agregado (Nivel Producto)

Este es el paso core de la Fase B. Reduciremos la granularidad de Tienda-Producto a solo Producto.

**Métricas a calcular:**
1.  `Total_Sales`: Suma de ventas (¿Qué tanto volumen mueve este producto en total?)
2.  `Avg_Sales`: Promedio de ventas por tienda (¿Qué tan bien performa individualmente?)
3.  `Store_Count`: Conteo único de tiendas (¿Qué tanta penetración de mercado tiene?)
4.  `Avg_MRP`: Precio promedio (Indica su gama: económico vs premium).
5.  `Avg_Visibility`: Visibilidad promedio.
6.  `Item_Weight`: Promedio (que será igual al valor único).
7.  Variables Categóricas: Tomaremos el "primero" ya que son constantes por producto.

In [5]:
# Definir diccionario de agregaciones
aggs = {
    'Item_Outlet_Sales': ['sum', 'mean'],      # Total Volumen y Rendimiento Promedio
    'Outlet_Identifier': 'nunique',            # Store Count
    'Item_MRP': 'mean',                        # Precio Promedio
    'Item_Visibility': 'mean',                 # Visibilidad Promedio
    'Item_Weight': 'first',                    # Peso (Constante)
    'Item_Fat_Content': 'first',               # Categórica (Constante)
    'Broad_Category': 'first',                 # Categórica (Constante)
    'Item_Type': 'first'                       # Categórica (Constante - opcional pero útil)
}

# Agrupar
df_product = df.groupby('Item_Identifier').agg(aggs).reset_index()

# Aplanar los nombres de columnas (MultiIndex a Single Index)
df_product.columns = [
    'Item_Identifier', 
    'Total_Sales', 
    'Avg_Sales', 
    'Store_Count', 
    'Avg_MRP', 
    'Avg_Visibility', 
    'Item_Weight', 
    'Item_Fat_Content', 
    'Broad_Category', 
    'Item_Type'
]

print(f"Dimensiones del dataset agregado: {df_product.shape}")
print(f"Debería ser (1559, 10). Resultado: {df_product.shape}")
df_product.head()

Dimensiones del dataset agregado: (1559, 10)
Debería ser (1559, 10). Resultado: (1559, 10)


,Item_Identifier,Total_Sales,Avg_Sales,Store_Count,Avg_MRP,Avg_Visibility,Item_Weight,Item_Fat_Content,Broad_Category,Item_Type
0,DRA12,11061.6012,1843.600200,6,141.865400,0.047934,11.600,Low Fat,Drinks,Soft Drinks
1,DRA24,15723.5328,2246.218971,7,164.086800,0.048062,19.350,Regular,Drinks,Soft Drinks
2,DRA59,20915.4412,2614.430150,8,185.179900,0.153963,8.270,Regular,Drinks,Soft Drinks
3,DRB01,4554.0720,1518.024000,3,189.586333,0.082126,7.390,Low Fat,Drinks,Soft Drinks
4,DRB13,12144.1920,2428.838400,5,189.693000,0.008002,6.115,Regular,Drinks,Soft Drinks


---
## 4. Preprocesamiento para Clustering (Encoding y Scaling)

Generaremos dos datasets:
1.  **Interpretación:** Mantiene los valores originales.
2.  **Modelado:** Numérico y escalado, listo para K-Means.

### 4.1 Encoding
Transformaremos `Item_Fat_Content` y `Broad_Category`. No usaremos `Item_Type` para el clustering inicial para evitar la maldición de la dimensionalidad (muchas columnas dummy), usaremos `Broad_Category` como proxy general.

In [6]:
# Copia para modelado
df_modeling = df_product.copy().set_index('Item_Identifier')

# 1. Drop de columnas que no usaremos en el modelo numérico estricto
# Item_Type es muy granular, Broad_Category captura la esencia mejor para clustering de alto nivel
df_modeling = df_modeling.drop(columns=['Item_Type'])

# 2. One Hot Encoding para categoricas
df_modeling = pd.get_dummies(df_modeling, columns=['Item_Fat_Content', 'Broad_Category'], drop_first=False)

# Visualizar columnas resultantes
print(df_modeling.columns)

Index(['Total_Sales', 'Avg_Sales', 'Store_Count', 'Avg_MRP', 'Avg_Visibility',
       'Item_Weight', 'Item_Fat_Content_Low Fat',
       'Item_Fat_Content_Non-Edible', 'Item_Fat_Content_Regular',
       'Broad_Category_Drinks', 'Broad_Category_Food',
       'Broad_Category_Non-Consumable'],
      dtype='object')


### 4.2 Scaling
Los algoritmos de clustering basados en distancia (como K-Means) son sensibles a la magnitud. `Total_Sales` (miles) dominaría sobre `Item_Visibility` (0.01). Usaremos `StandardScaler`.

In [7]:
scaler = StandardScaler()

# Escalamos todo el dataframe de modelado
df_modeling_scaled = pd.DataFrame(scaler.fit_transform(df_modeling), 
                                  columns=df_modeling.columns, 
                                  index=df_modeling.index)

df_modeling_scaled.head(3)

,Total_Sales,Avg_Sales,Store_Count,Avg_MRP,Avg_Visibility,Item_Weight,Item_Fat_Content_Low Fat,Item_Fat_Content_Non-Edible,Item_Fat_Content_Regular,Broad_Category_Drinks,Broad_Category_Food,Broad_Category_Non-Consumable
Item_Identifier,,,,,,,,,,,,
DRA12,-0.122721,-0.308001,0.348838,0.013769,-0.472216,-0.260236,1.089282,-0.4831,-0.739342,3.122775,-1.594736,-0.4831
DRA24,0.539887,0.047767,1.003278,0.371701,-0.469498,1.408344,-0.918036,-0.4831,1.352554,3.122775,-1.594736,-0.4831
DRA59,1.277820,0.373131,1.657717,0.711459,1.779100,-0.977187,-0.918036,-0.4831,1.352554,3.122775,-1.594736,-0.4831


---
## 5. Exportación y Resumen

Guardaremos los archivos en `../Data/Processed/`.

In [8]:
# Guardar Dataset para Interpretación (Valores Reales)
df_product.to_csv('../Data/Processed/product_level_interpretation_gemini.csv', index=False)

# Guardar Dataset para Modelado (Escalado y Encodeado)
df_modeling_scaled.to_csv('../Data/Processed/product_level_modeling_gemini.csv', index=True)

print("Archivos guardados exitosamente.")
print("1. product_level_interpretation.csv (Para análisis y dashboard)")
print("2. product_level_modeling.csv (Para algoritmos de clustering)")

Archivos guardados exitosamente.
1. product_level_interpretation.csv (Para análisis y dashboard)
2. product_level_modeling.csv (Para algoritmos de clustering)


### Resumen de Transformaciones Aplicadas

1.  **Limpieza:**
    * Unificación de etiquetas `Item_Fat_Content`.
    * Imputación de `Item_Weight` basada en ID (lógica de producto único).
    * Imputación de `Item_Visibility` basada en promedio del producto (corrigiendo ceros).

2.  **Ingeniería:**
    * Creación de `Broad_Category` (Food, Drink, Non-Consumable).
    * Corrección semántica: Non-Consumable ahora es 'Non-Edible' en contenido de grasa.

3.  **Agregación:**
    * Reducción de 8523 filas a **1559 productos únicos**.
    * Nuevas métricas generadas: `Store_Count`, `Total_Sales`, `Avg_Visibility`, etc.

4.  **Preprocesamiento:**
    * One-Hot Encoding aplicado a categorías.
    * Standard Scaling aplicado para normalizar rangos de ventas vs visibilidad.

**Siguiente Paso (Fase C):** Cargar `product_level_modeling.csv`, determinar el número óptimo de clusters (Elbow Method/Silhouette) y ejecutar K-Means.